In [ ]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install datasets loralib sentencepiece
!pip -q install pypdf

!pip install torch
!pip -q install sentence_transformers
!pip install chromadb
!pip install openai
!pip install tiktoken
!pip install -U langchain-community
!pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 49.0 MB/s eta 0:00:0000:0100:01


In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter
# from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from huggingface_hub import notebook_login
import torch
import transformers

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI

import os
import sys

/workspaces/LLMDrivenOntologyDev/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
directory_path = '/content/drive/MyDrive/Wiki'

files = os.listdir(directory_path)

#OpenAI API Key stored as codespace secret

llm=ChatOpenAI(temperature=0, model_name='gpt-4')

allmodule_file_path = "/content/drive/MyDrive/ModulesR/RR/WB_Schema_Relationships.txt"
m_path = "/content/drive/MyDrive/ModulesR/"

modules_files = os.listdir(m_path)

### All modules
with open(allmodule_file_path, "r", encoding="utf-8") as module_file:
        module_content = module_file.read()
### Module by module
module_content1={}
for f in modules_files:
  if f.endswith(".txt"):
    m_path1 = m_path+f

    with open(m_path1, "r", encoding="utf-8") as module_file:
       file_name = f.replace(".txt", "")
       module_content1[file_name] = module_file.read()

In [ ]:
##gpt

file = files[0] # to be edited for each file
document=[]
if file.endswith(".pdf"):
  pdf_path=""+file
  loader=PyPDFLoader(pdf_path)
  document.extend(loader.load())
elif file.endswith('.docx') or file.endswith('.doc'):
  doc_path=""+file
  loader=Docx2txtLoader(doc_path)
  document.extend(loader.load())
elif file.endswith('.txt'):

    dir_path = os.path.join(directory_path, file)
    text_path = dir_path
    loader=TextLoader(text_path, encoding = 'UTF-8')
    document.extend(loader.load())
    document_splitter= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"])
    document_chunks=document_splitter.split_documents(document)
    # embeddings = OpenAIEmbeddings()
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vectordb=Chroma.from_documents(document_chunks,embedding=embeddings, persist_directory='./data')
    vectordb.persist()
    # llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})
    memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True,output_key='answer')
    pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm,
                                              retriever=vectordb.as_retriever(search_kwargs={'k':10}),
                                              verbose=False, memory=memory,return_source_documents=True)
    # for j in module_content1:
    result=pdf_qa({"question":"Retrieve the related parts of the document that match with this ontology module, modules:"+str(module_content)+ "and populate the ontology based on the text in the form of the module relations like:  Name Record Module:1. has_Name(Agent, Name): has_Name(Joseph Vance Lewis, 'Joseph Vance Lewis') 2. has_Surname(Agent, Surname): has_Surname(Joseph Vance Lewis, 'Lewis') 3. has_First_Name(Agent, First_Name): has_First_Name(Joseph Vance Lewis, 'Joseph').Interagent Relationship Record Module: 1. has_Interagent_Relationship_Type_To(Agent, Relationship_Type): has_Interagent_Relationship_Type_To(Joseph Vance Lewis, 'Enslaver') 2. is_Relationship_To(Agent, Agent): is_Relationship_To(Joseph Vance Lewis, 'Bwril Nate'). Note: The Interagent Relationship Record Module describes the relationship to another agent who is an enslaver or owner. So use the keywords Enslaver or Owner to describe the relationship type. Sex Record Module: 1. has_Sex(Agent, Sex_Type): has_Sex(Joseph Vance Lewis, 'Male'). Occupation Record Module: 1. has_Occupation(Agent, Occupation): has_Occupation(Joseph Vance Lewis, 'Slave, Educator, Lawyer, and Autobiographer'). Age Record Module: 1. has_Age(Agent, Age): has_Age(Joseph Vance Lewis, 72) 2. has_BirthDate(Agent, Date_of_Birth): has_BirthDate(Joseph Vance Lewis, 'December 25, 1853') 3. has_DeathDate(Agent, Date_of_Death): has_DeathDate(Joseph Vance Lewis, 'April 24, 1925'). Person Status Record Module: 1. has_Person_Status (Agent, Person_Status): has_Person_Status(Joseph Vance Lewis, 'Enslaved Person, Lawyer') 2. has_Status_Generating_Event(Person_Status, Event): has_Status_Generating_Event(Enslaved Person, 'Sale of estate of Joseph Dubreuil'), has_Status_Generating_Event(Lawyer, 'Emancipation') 3. recorded_At(Person_Status_information, Event): recorded_At(Enslaved Person, 'Sale of estate of Joseph Dubreuil'), recorded_At(Lawyer, 'Emancipation'). Race Record Module: has_Race_or_Color(Agent, Race_or_Color): has_Race_or_Color(Joseph Vance Lewis, African-American).  Note: Skip the relations for which there is no information listed in the text file.The birth and death dates are mentioned in parenthesis after the agent name. For example: Joseph Vance Lewis (December 25, 1853 – April 24, 1925), was a slave who was freed. Here the Birth date is December 25, 1853 and Death date is April 24, 1925."})
    summ = result['answer']

    source_documents = result.get("source_documents", [])
    sen=""
    for doc in source_documents:
        sen = sen + str(doc.page_content)
        sen = sen + "\n"
    f_name = os.path.join("/content/drive/MyDrive/Wiki/NewResponses/GPT4_WB_notrestrictedToMAgent",os.path.splitext(file)[0])
    if not os.path.exists(f_name):
      os.makedirs(f_name, exist_ok=True)
    # response_file_path = os.path.join(f_name, f"{os.path.splitext(file)[0]}_"f"{j}_retrieved.txt")
    # response_file_path1 = os.path.join(f_name, f"{os.path.splitext(file)[0]}_"f"{j}_triples.txt")
    response_file_path = os.path.join(f_name, f"{os.path.splitext(file)[0]}_retrieved.txt")
    response_file_path1 = os.path.join(f_name, f"{os.path.splitext(file)[0]}_triples.txt")
    with open(response_file_path1, "w", encoding="utf-8") as response_file1:
        response_file1.write(summ)
    with open(response_file_path, "w", encoding="utf-8") as response_file:
        response_file.write(sen)
    torch.cuda.empty_cache()
    del result
    del summ
    del source_documents
    del sen
    memory.clear()







In [ ]:
f = files[139]

In [ ]:
##gpt

file = f
document=[]
if file.endswith(".pdf"):
  pdf_path=""+file
  loader=PyPDFLoader(pdf_path)
  document.extend(loader.load())
elif file.endswith('.docx') or file.endswith('.doc'):
  doc_path=""+file
  loader=Docx2txtLoader(doc_path)
  document.extend(loader.load())
elif file.endswith('.txt'):

    dir_path = os.path.join(directory_path, file)
    text_path = dir_path
    loader=TextLoader(text_path, encoding = 'UTF-8')
    document.extend(loader.load())
    document_splitter= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"])
    document_chunks=document_splitter.split_documents(document)
    # embeddings = OpenAIEmbeddings()
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vectordb=Chroma.from_documents(document_chunks,embedding=embeddings, persist_directory='./data')
    vectordb.persist()
    # llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})
    memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True,output_key='answer')
    pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm,
                                              retriever=vectordb.as_retriever(search_kwargs={'k':10}),
                                              verbose=False, memory=memory,return_source_documents=True)
    # for j in module_content1:
    result=pdf_qa({"question":"Retrieve the related parts of the document that match with this ontology module, modules:"+str(module_content)+ "and populate the ontology based on the text in the form of the module relations like:  Name Record Module:1. has_Name(Agent, Name): has_Name(Joseph Vance Lewis, 'Joseph Vance Lewis') 2. has_Surname(Agent, Surname): has_Surname(Joseph Vance Lewis, 'Lewis') 3. has_First_Name(Agent, First_Name): has_First_Name(Joseph Vance Lewis, 'Joseph').Interagent Relationship Record Module: 1. has_Interagent_Relationship_Type_To(Agent, Relationship_Type): has_Interagent_Relationship_Type_To(Joseph Vance Lewis, 'Enslaver') 2. is_Relationship_To(Agent, Agent): is_Relationship_To(Joseph Vance Lewis, 'Bwril Nate'). Note: The Interagent Relationship Record Module describes the relationship to another agent who is an enslaver or owner. So use the keywords Enslaver or Owner to describe the relationship type. Sex Record Module: 1. has_Sex(Agent, Sex_Type): has_Sex(Joseph Vance Lewis, 'Male'). Occupation Record Module: 1. has_Occupation(Agent, Occupation): has_Occupation(Joseph Vance Lewis, 'Slave, Educator, Lawyer, and Autobiographer'). Age Record Module: 1. has_Age(Agent, Age): has_Age(Joseph Vance Lewis, 72) 2. has_BirthDate(Agent, Date_of_Birth): has_BirthDate(Joseph Vance Lewis, 'December 25, 1853') 3. has_DeathDate(Agent, Date_of_Death): has_DeathDate(Joseph Vance Lewis, 'April 24, 1925'). Person Status Record Module: 1. has_Person_Status (Agent, Person_Status): has_Person_Status(Joseph Vance Lewis, 'Enslaved Person, Lawyer') 2. has_Status_Generating_Event(Person_Status, Event): has_Status_Generating_Event(Enslaved Person, 'Sale of estate of Joseph Dubreuil'), has_Status_Generating_Event(Lawyer, 'Emancipation') 3. recorded_At(Person_Status_information, Event): recorded_At(Enslaved Person, 'Sale of estate of Joseph Dubreuil'), recorded_At(Lawyer, 'Emancipation'). Race Record Module: has_Race_or_Color(Agent, Race_or_Color): has_Race_or_Color(Joseph Vance Lewis, African-American).  Note: Skip the relations for which there is no information listed in the text file.The birth and death dates are mentioned in parenthesis after the agent name. For example: Joseph Vance Lewis (December 25, 1853 – April 24, 1925), was a slave who was freed. Here the Birth date is December 25, 1853 and Death date is April 24, 1925."})
    summ = result['answer']

    source_documents = result.get("source_documents", [])
    sen=""
    for doc in source_documents:
        sen = sen + str(doc.page_content)
        sen = sen + "\n"
    f_name = os.path.join("/content/drive/MyDrive/Wiki/NewResponses/GPT4_WB_notrestrictedToMAgent",os.path.splitext(file)[0])
    if not os.path.exists(f_name):
      os.makedirs(f_name, exist_ok=True)
    # response_file_path = os.path.join(f_name, f"{os.path.splitext(file)[0]}_"f"{j}_retrieved.txt")
    # response_file_path1 = os.path.join(f_name, f"{os.path.splitext(file)[0]}_"f"{j}_triples.txt")
    response_file_path = os.path.join(f_name, f"{os.path.splitext(file)[0]}_retrieved.txt")
    response_file_path1 = os.path.join(f_name, f"{os.path.splitext(file)[0]}_triples.txt")
    with open(response_file_path1, "w", encoding="utf-8") as response_file1:
        response_file1.write(summ)
    with open(response_file_path, "w", encoding="utf-8") as response_file:
        response_file.write(sen)
    torch.cuda.empty_cache()
    del result
    del summ
    del source_documents
    del sen
    memory.clear()

In [ ]:
#gpt

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
vectordb=Chroma.from_documents(document_chunks,embedding=embeddings, persist_directory='./data')
vectordb.persist()
# llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})
memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True,output_key='answer')
pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm,
                                          retriever=vectordb.as_retriever(search_kwargs={'k':10}),
                                          verbose=False, memory=memory,return_source_documents=True)
# for j in module_content1:
result=pdf_qa({"question":"Retrieve the related parts of the document that match with this ontology module that is related just for an agent that the document starts with the name, modules:"+str(module_content)+ "and populate the ontology based on the text in the form of the module relations like:  Name Record Module:1. has_Name(Agent, Name): has_Name(Joseph Vance Lewis, 'Joseph Vance Lewis') 2. has_Surname(Agent, Surname): has_Surname(Joseph Vance Lewis, 'Lewis') 3. has_First_Name(Agent, First_Name): has_First_Name(Joseph Vance Lewis, 'Joseph').Interagent Relationship Record Module: 1. has_Interagent_Relationship_Type_To(Agent, Relationship_Type): has_Interagent_Relationship_Type_To(Joseph Vance Lewis, 'Enslaver') 2. is_Relationship_To(Agent, Agent): is_Relationship_To(Joseph Vance Lewis, 'Bwril Nate'). Note: The Interagent Relationship Record Module describes the relationship to another agent who is an enslaver or owner. So use the keywords Enslaver or Owner to describe the relationship type. Sex Record Module: 1. has_Sex(Agent, Sex_Type): has_Sex(Joseph Vance Lewis, 'Male'). Occupation Record Module: 1. has_Occupation(Agent, Occupation): has_Occupation(Joseph Vance Lewis, 'Slave, Educator, Lawyer, and Autobiographer'). Age Record Module: 1. has_Age(Agent, Age): has_Age(Joseph Vance Lewis, 72) 2. has_BirthDate(Agent, Date_of_Birth): has_BirthDate(Joseph Vance Lewis, 'December 25, 1853') 3. has_DeathDate(Agent, Date_of_Death): has_DeathDate(Joseph Vance Lewis, 'April 24, 1925'). Person Status Record Module: 1. has_Person_Status (Agent, Person_Status): has_Person_Status(Joseph Vance Lewis, 'Enslaved Person, Lawyer') 2. has_Status_Generating_Event(Person_Status, Event): has_Status_Generating_Event(Enslaved Person, 'Sale of estate of Joseph Dubreuil'), has_Status_Generating_Event(Lawyer, 'Emancipation') 3. recorded_At(Person_Status_information, Event): recorded_At(Enslaved Person, 'Sale of estate of Joseph Dubreuil'), recorded_At(Lawyer, 'Emancipation'). Race Record Module: has_Race_or_Color(Agent, Race_or_Color): has_Race_or_Color(Joseph Vance Lewis, African-American).  Note: Skip the relations for which there is no information listed in the text file.The birth and death dates are mentioned in parenthesis after the agent name. For example: Joseph Vance Lewis (December 25, 1853 – April 24, 1925), was a slave who was freed. Here the Birth date is December 25, 1853 and Death date is April 24, 1925."})
summ = result['answer']

source_documents = result.get("source_documents", [])
sen=""
for doc in source_documents:
    sen = sen + str(doc.page_content)
    sen = sen + "\n"
f_name = os.path.join("/content/drive/MyDrive/Wiki/NewResponses/GPT4_WB_MainAgent",os.path.splitext(file)[0])
if not os.path.exists(f_name):
  os.makedirs(f_name, exist_ok=True)
# response_file_path = os.path.join(f_name, f"{os.path.splitext(file)[0]}_"f"{j}_retrieved.txt")
# response_file_path1 = os.path.join(f_name, f"{os.path.splitext(file)[0]}_"f"{j}_triples.txt")
response_file_path = os.path.join(f_name, f"{os.path.splitext(file)[0]}_retrieved.txt")
response_file_path1 = os.path.join(f_name, f"{os.path.splitext(file)[0]}_triples.txt")
with open(response_file_path1, "w", encoding="utf-8") as response_file1:
    response_file1.write(summ)
with open(response_file_path, "w", encoding="utf-8") as response_file:
    response_file.write(sen)
torch.cuda.empty_cache()
del result
del summ
del source_documents
del sen
memory.clear()







In [ ]:
from huggingface_hub import login

login('Hugging face token',add_to_git_credential=True)

In [ ]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

llm1 = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    # cache_dir="/data/yash/base_models",
    device_map='auto'
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct",
                                          # cache_dir="/data/yash/base_models"
                                         )

In [ ]:
from langchain import HuggingFacePipeline
from transformers import pipeline
import torch
pipe=pipeline("text-generation",
              model=llm1,
              tokenizer=tokenizer,
              torch_dtype=torch.bfloat16,
              device_map='auto',
              max_new_tokens=2048,
              min_new_tokens=-1,
              top_k=30

              )

In [ ]:
llm11 =HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [ ]:
#llama3
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
vectordb=Chroma.from_documents(document_chunks,embedding=embeddings, persist_directory='./data')
vectordb.persist()
# llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})
memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True,output_key='answer')
pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm11,
                                          retriever=vectordb.as_retriever(search_kwargs={'k':10}),
                                          verbose=False, memory=memory,return_source_documents=True)
# for j in module_content1:
result=pdf_qa({"question":"Retrieve the related parts of the document that match with this ontology module that is related just for an agent that the document starts with the name, modules:"+str(module_content)+ "and populate the ontology based on the text in the form of the module relations like:  Name Record Module:1. has_Name(Agent, Name): has_Name(Joseph Vance Lewis, 'Joseph Vance Lewis') 2. has_Surname(Agent, Surname): has_Surname(Joseph Vance Lewis, 'Lewis') 3. has_First_Name(Agent, First_Name): has_First_Name(Joseph Vance Lewis, 'Joseph').Interagent Relationship Record Module: 1. has_Interagent_Relationship_Type_To(Agent, Relationship_Type): has_Interagent_Relationship_Type_To(Joseph Vance Lewis, 'Enslaver') 2. is_Relationship_To(Agent, Agent): is_Relationship_To(Joseph Vance Lewis, 'Bwril Nate'). Note: The Interagent Relationship Record Module describes the relationship to another agent who is an enslaver or owner. So use the keywords Enslaver or Owner to describe the relationship type. Sex Record Module: 1. has_Sex(Agent, Sex_Type): has_Sex(Joseph Vance Lewis, 'Male'). Occupation Record Module: 1. has_Occupation(Agent, Occupation): has_Occupation(Joseph Vance Lewis, 'Slave, Educator, Lawyer, and Autobiographer'). Age Record Module: 1. has_Age(Agent, Age): has_Age(Joseph Vance Lewis, 72) 2. has_BirthDate(Agent, Date_of_Birth): has_BirthDate(Joseph Vance Lewis, 'December 25, 1853') 3. has_DeathDate(Agent, Date_of_Death): has_DeathDate(Joseph Vance Lewis, 'April 24, 1925'). Person Status Record Module: 1. has_Person_Status (Agent, Person_Status): has_Person_Status(Joseph Vance Lewis, 'Enslaved Person, Lawyer') 2. has_Status_Generating_Event(Person_Status, Event): has_Status_Generating_Event(Enslaved Person, 'Sale of estate of Joseph Dubreuil'), has_Status_Generating_Event(Lawyer, 'Emancipation') 3. recorded_At(Person_Status_information, Event): recorded_At(Enslaved Person, 'Sale of estate of Joseph Dubreuil'), recorded_At(Lawyer, 'Emancipation'). Race Record Module: has_Race_or_Color(Agent, Race_or_Color): has_Race_or_Color(Joseph Vance Lewis, African-American).  Note: Skip the relations for which there is no information listed in the text file.The birth and death dates are mentioned in parenthesis after the agent name. For example: Joseph Vance Lewis (December 25, 1853 – April 24, 1925), was a slave who was freed. Here the Birth date is December 25, 1853 and Death date is April 24, 1925."})
summ = result['answer']
start_pos = summ.find("Helpful Answer:")

# Extract the text after "Helpful Answer:"
if start_pos != -1:
    helpful_answer_text = summ[start_pos + len("Helpful Answer:"):].strip()
else:
    helpful_answer_text = ""

# print(helpful_answer_text)


source_documents = result.get("source_documents", [])
sen=""
for doc in source_documents:
    sen = sen + str(doc.page_content)
    sen = sen + "\n"
f_name = os.path.join("/content/drive/MyDrive/Wiki/NewResponses/llama_WB_MainAgent",os.path.splitext(file)[0])
if not os.path.exists(f_name):
  os.makedirs(f_name, exist_ok=True)
# response_file_path = os.path.join(f_name, f"{os.path.splitext(file)[0]}_"f"{j}_retrieved.txt")
# response_file_path1 = os.path.join(f_name, f"{os.path.splitext(file)[0]}_"f"{j}_triples.txt")
response_file_path = os.path.join(f_name, f"{os.path.splitext(file)[0]}_retrieved.txt")
response_file_path1 = os.path.join(f_name, f"{os.path.splitext(file)[0]}_triples.txt")
with open(response_file_path1, "w", encoding="utf-8") as response_file1:
    response_file1.write(helpful_answer_text)
with open(response_file_path, "w", encoding="utf-8") as response_file:
    response_file.write(sen)
torch.cuda.empty_cache()
del result
del summ
del helpful_answer_text
del source_documents
del sen
memory.clear()


In [ ]:

# llama3
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
vectordb=Chroma.from_documents(document_chunks,embedding=embeddings, persist_directory='./data')
vectordb.persist()
# llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})
memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True,output_key='answer')
pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm11,
                                          retriever=vectordb.as_retriever(search_kwargs={'k':10}),
                                          verbose=False, memory=memory,return_source_documents=True)
# for j in module_content1:
result=pdf_qa({"question":"Retrieve the related parts of the document that match with this ontology module that is related just for an agent that the document starts with the name, modules:"+str(module_content)+ "and populate the ontology based on the text in the form of the module relations like:  Name Record Module:1. has_Name(Agent, Name): has_Name(Joseph Vance Lewis, 'Joseph Vance Lewis') 2. has_Surname(Agent, Surname): has_Surname(Joseph Vance Lewis, 'Lewis') 3. has_First_Name(Agent, First_Name): has_First_Name(Joseph Vance Lewis, 'Joseph').Interagent Relationship Record Module: 1. has_Interagent_Relationship_Type_To(Agent, Relationship_Type): has_Interagent_Relationship_Type_To(Joseph Vance Lewis, 'Enslaver') 2. is_Relationship_To(Agent, Agent): is_Relationship_To(Joseph Vance Lewis, 'Bwril Nate'). Note: The Interagent Relationship Record Module describes the relationship to another agent who is an enslaver or owner. So use the keywords Enslaver or Owner to describe the relationship type. Sex Record Module: 1. has_Sex(Agent, Sex_Type): has_Sex(Joseph Vance Lewis, 'Male'). Occupation Record Module: 1. has_Occupation(Agent, Occupation): has_Occupation(Joseph Vance Lewis, 'Slave, Educator, Lawyer, and Autobiographer'). Age Record Module: 1. has_Age(Agent, Age): has_Age(Joseph Vance Lewis, 72) 2. has_BirthDate(Agent, Date_of_Birth): has_BirthDate(Joseph Vance Lewis, 'December 25, 1853') 3. has_DeathDate(Agent, Date_of_Death): has_DeathDate(Joseph Vance Lewis, 'April 24, 1925'). Person Status Record Module: 1. has_Person_Status (Agent, Person_Status): has_Person_Status(Joseph Vance Lewis, 'Enslaved Person, Lawyer') 2. has_Status_Generating_Event(Person_Status, Event): has_Status_Generating_Event(Enslaved Person, 'Sale of estate of Joseph Dubreuil'), has_Status_Generating_Event(Lawyer, 'Emancipation') 3. recorded_At(Person_Status_information, Event): recorded_At(Enslaved Person, 'Sale of estate of Joseph Dubreuil'), recorded_At(Lawyer, 'Emancipation'). Race Record Module: has_Race_or_Color(Agent, Race_or_Color): has_Race_or_Color(Joseph Vance Lewis, African-American).  Note: Skip the relations for which there is no information listed in the text file.The birth and death dates are mentioned in parenthesis after the agent name. For example: Joseph Vance Lewis (December 25, 1853 – April 24, 1925), was a slave who was freed. Here the Birth date is December 25, 1853 and Death date is April 24, 1925."})
summ = result['answer']
start_pos = summ.find("Helpful Answer:")

# Extract the text after "Helpful Answer:"
if start_pos != -1:
    helpful_answer_text = summ[start_pos + len("Helpful Answer:"):].strip()
else:
    helpful_answer_text = ""


source_documents = result.get("source_documents", [])
sen=""
for doc in source_documents:
    sen = sen + str(doc.page_content)
    sen = sen + "\n"
f_name = os.path.join("/content/drive/MyDrive/Wiki/NewResponses/llama_WB_notrestrictedToMAgent",os.path.splitext(file)[0])
if not os.path.exists(f_name):
  os.makedirs(f_name, exist_ok=True)
# response_file_path = os.path.join(f_name, f"{os.path.splitext(file)[0]}_"f"{j}_retrieved.txt")
# response_file_path1 = os.path.join(f_name, f"{os.path.splitext(file)[0]}_"f"{j}_triples.txt")
response_file_path = os.path.join(f_name, f"{os.path.splitext(file)[0]}_retrieved.txt")
response_file_path1 = os.path.join(f_name, f"{os.path.splitext(file)[0]}_triples.txt")
with open(response_file_path1, "w", encoding="utf-8") as response_file1:
    response_file1.write(helpful_answer_text)
with open(response_file_path, "w", encoding="utf-8") as response_file:
    response_file.write(sen)
torch.cuda.empty_cache()
del result
del summ
del helpful_answer_text
del source_documents
del sen
memory.clear()